In [1]:
import pandas  as pd

df = pd.read_csv("../Data/1_binary_landscapes.txt", header=None)

df = df.drop(0)

df.reset_index(drop=True, inplace=True)

df.head()

,0
0,L 21 ar5fv a99am a5nnd aj4f awzgl a6zkl acz3g ...
1,L 9 apvh8 a7fgq a6p61 a5u7p awzsw akw11 alhla ...
2,L 15 ajj0r ak3r a5pgg af36u avwcg an4hp a0r1w ...
3,L 15 a4ra4 a48r2 a0h5n abzhk a52uu au895 afv2 ...
4,L 15 a5rbj a8z5v a46vk a9h9q a98wh ajbc a7w5h ...


In [2]:
def transform_data(row, idx):
    parts = row.split()
    return {
        "id": [idx],
        "type": parts[0],
        "tagNumber": int(parts[1]),
        "tags": parts[2:]
    }

df_transformed = df.apply(lambda row: transform_data(row[0], row.name), axis=1)
df_transformed = pd.DataFrame(df_transformed.tolist())
df_transformed.head()

,id,type,tagNumber,tags
0,[0],L,21,"[ar5fv, a99am, a5nnd, aj4f, awzgl, a6zkl, acz3..."
1,[1],L,9,"[apvh8, a7fgq, a6p61, a5u7p, awzsw, akw11, alh..."
2,[2],L,15,"[ajj0r, ak3r, a5pgg, af36u, avwcg, an4hp, a0r1..."
3,[3],L,15,"[a4ra4, a48r2, a0h5n, abzhk, a52uu, au895, afv..."
4,[4],L,15,"[a5rbj, a8z5v, a46vk, a9h9q, a98wh, ajbc, a7w5..."


In [3]:
df_L = df_transformed[df_transformed['type'] == 'L']
df_P = df_transformed[df_transformed['type'] == 'P']

In [4]:
import numpy as np
if len(df_P) < 2:
    # raise ValueError("The length of df_P is less than 2. Cannot proceed with the operation.")
    print("The length of df_P is less than 2. Cannot proceed with the operation.")
else:
    # Create a common_tags_matrix for demonstration purposes
    common_tags_matrix = np.random.randint(0, 10, size=(len(df_P), len(df_P)))

    # Define max_common_tags for demonstration purposes
    max_common_tags = np.max(common_tags_matrix)

    # Flatten the common_tags_matrix and get the indices
    flat_indices = np.argsort(common_tags_matrix, axis=None, kind='stable')

    # Convert the flat indices back to 2D indices
    row_indices, col_indices = np.unravel_index(flat_indices, common_tags_matrix.shape)

    # Get the pairs with the highest common tags, ensuring each row is used only once
    used_indices = set()
    highest_common_pairs = []
    for i in range(len(row_indices)):
        if row_indices[i] != col_indices[i]:  # Ensure pairs are not made with themselves
            if common_tags_matrix[row_indices[i], col_indices[i]] == max_common_tags:
                if row_indices[i] not in used_indices and col_indices[i] not in used_indices:
                    highest_common_pairs.append((row_indices[i], col_indices[i]))
                    used_indices.add(row_indices[i])
                    used_indices.add(col_indices[i])

    # If there are unused rows, pair them with any other unused row
    unused_indices = [idx for idx in range(len(df_P)) if idx not in used_indices]
    while len(unused_indices) > 1:
        highest_common_pairs.append((unused_indices.pop(0), unused_indices.pop(0)))

    # Display the pairs
    print("Pairs of rows with the highest common tags in df_P (each row used only once):")
    print(highest_common_pairs)

The length of df_P is less than 2. Cannot proceed with the operation.


In [5]:

if len(df_P) < 2:
    print("The length of df_P is less than 2. Cannot proceed with the operation.")
    # raise ValueError("The length of df_P is less than 2. Cannot proceed with the operation.")
else:
    merged_rows = []
    for pair in highest_common_pairs:
        row1 = df_P.iloc[pair[0]]
        row2 = df_P.iloc[pair[1]]
        merged_id = [row1['id'][0], row2['id'][0]]
        merged_tags = list(set(row1['tags'] + row2['tags']))
        tag_number = len(merged_tags)
        type = "P"
        merged_rows.append({"id": merged_id, "tags": merged_tags, "tagNumber": tag_number, "type":type})

    df_merged = pd.DataFrame(merged_rows)
    df_merged.head()

The length of df_P is less than 2. Cannot proceed with the operation.


In [6]:
if len(df_P) < 2:
    final_df = pd.concat([df_P, df_L], ignore_index=True)
else:
    final_df = pd.concat([df_merged, df_L], ignore_index=True)
final_df.head()

,id,type,tagNumber,tags
0,[0],L,21,"[ar5fv, a99am, a5nnd, aj4f, awzgl, a6zkl, acz3..."
1,[1],L,9,"[apvh8, a7fgq, a6p61, a5u7p, awzsw, akw11, alh..."
2,[2],L,15,"[ajj0r, ak3r, a5pgg, af36u, avwcg, an4hp, a0r1..."
3,[3],L,15,"[a4ra4, a48r2, a0h5n, abzhk, a52uu, au895, afv..."
4,[4],L,15,"[a5rbj, a8z5v, a46vk, a9h9q, a98wh, ajbc, a7w5..."


In [7]:
final_df.shape

(80000, 4)

In [8]:
invalid_ids = final_df[(final_df['type'] == 'P') & (final_df['id'].apply(len) == 2)]
print("Rows with type 'P' where id array does not have exactly two values:")
print(len(invalid_ids))

Rows with type 'P' where id array does not have exactly two values:
0


In [12]:
df_L.head()
df_P.head()
final_df.head()

,id,type,tagNumber,tags
0,[0],L,21,"[ar5fv, a99am, a5nnd, aj4f, awzgl, a6zkl, acz3..."
1,[1],L,9,"[apvh8, a7fgq, a6p61, a5u7p, awzsw, akw11, alh..."
2,[2],L,15,"[ajj0r, ak3r, a5pgg, af36u, avwcg, an4hp, a0r1..."
3,[3],L,15,"[a4ra4, a48r2, a0h5n, abzhk, a52uu, au895, afv..."
4,[4],L,15,"[a5rbj, a8z5v, a46vk, a9h9q, a98wh, ajbc, a7w5..."


In [15]:
def min_form_number_of_common_tags(row1, row2):
    set1 = set(row1['tags'])
    set2 = set(row2['tags'])
    
    tags_in_row1_not_in_row2 = len(set1 - set2)
    tags_in_row2_not_in_row1 = len(set2 - set1)
    common_tags = len(set1 & set2)
    
    print(tags_in_row1_not_in_row2)
    
    return min(tags_in_row1_not_in_row2, tags_in_row2_not_in_row1, common_tags)

# Example usage:
row1 = final_df.iloc[0]
row2 = final_df.iloc[1]
result = min_form_number_of_common_tags(row1, row2)
print("Number of tags in first row not in second row:", result)

21
Number of tags in first row not in second row: 0


In [17]:
# Calculate the min_form_number_of_common_tags for each pair of rows
def calculate_min_form_number_of_common_tags(df):
    n = len(df)
    min_form_tags_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            min_form_tags_matrix[i, j] = min_form_number_of_common_tags(df.iloc[i], df.iloc[j])
            min_form_tags_matrix[j, i] = min_form_tags_matrix[i, j]
    return min_form_tags_matrix

min_form_tags_matrix = calculate_min_form_number_of_common_tags(final_df)

# Get the maximum value for each row
max_min_form_tags = np.max(min_form_tags_matrix, axis=1)

# Sort the dataframe based on the maximum min_form_number_of_common_tags values
sorted_indices = np.argsort(-max_min_form_tags)
sorted_final_df = final_df.iloc[sorted_indices].reset_index(drop=True)

sorted_final_df.head()

MemoryError: Unable to allocate 47.7 GiB for an array with shape (80000, 80000) and data type float64

In [ ]:
output_file_path = "../Data/output1.txt"

with open(output_file_path, 'w') as file:
    # Write the total number of rows
    file.write(f"{len(sorted_final_df)}\n")
    
    # Write the 'id' column of each row without brackets
    for idx, row in sorted_final_df.iterrows():
        ids_str = ' '.join(map(str, row['id']))
        file.write(f"{ids_str}\n")

print(f"Output file created at: {output_file_path}")

Output file created at: ../Data/output0.txt


In [ ]:
# # Example usage:
# row1 = sorted_final_df.iloc[12]
# row2 = sorted_final_df.iloc[13]
# result = min_form_number_of_common_tags(row1, row2)
# print("Number of tags in first row not in second row:", result)